In [45]:
## La version de python utilisée est 3.12.7

!pip install -r requirements.txt -q

In [46]:
import pandas as pd
import requests as rq
import lxml as lxml
from bs4 import BeautifulSoup
import io as io
import math
import gzip
import shutil
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import json
from pandasgui import show
import numpy as np
from io import BytesIO
from folium.plugins import HeatMap
import nbconvert

from script import process_data
from script import geolocaliser
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

In [47]:
# Charger le fichier SHP
gdf = gpd.read_file('data/zones_inondables/zones_inondables.shp')

# Afficher les premières lignes

# gdf_toulon = gdf[gdf['id_tri']=='FRD_TRI_TOULON'].drop(columns=['id_tri','dept','id'])
# gdf_est_var = gdf[gdf['id_tri']=='FRD_TRI_EST_VAR'].drop(columns=['id_tri','dept','id'])

C:\Users\flori\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pyogrio\raw.py:198: RuntimeWarning:

data/zones_inondables/zones_inondables.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.



In [48]:
base = pd.read_csv('data/transactions_with_batiment.csv',encoding='utf8',sep=";")

C:\Users\flori\AppData\Local\Temp\ipykernel_15692\164896197.py:1: DtypeWarning:

Columns (14,16) have mixed types. Specify dtype option on import or set low_memory=False.



In [49]:
# Étape 1 : Lister les noms des communes ayant au moins une "submersion marine"
communes_submersion = base.loc[base['libelle_type_inondation'] == "submersion marine", 'nom_commune'].unique()

# Étape 2 : Filtrer le DataFrame pour ne garder que :
# - Les communes ayant une submersion marine
# - Les communes où "identifiant_tri" est renseigné pour au moins une des lignes
base = base[
    base['nom_commune'].isin(communes_submersion) & 
    base.groupby('nom_commune')['identifiant_tri'].transform('any')
]


In [50]:
# Filtrer les valeurs uniques par 'nom_commune' et trier par 'population'
top_40_communes = base.drop_duplicates(subset='nom_commune').nlargest(40, 'Population')

# Afficher un aperçu des 40 plus grandes communes
print(top_40_communes[['nom_commune', 'Population']])


                   nom_commune  Population
8185                      Nice      348085
51978                 Bordeaux      261804
120229                  Toulon      180452
25617                     Caen      108200
91509                Dunkerque       86788
83682    Cherbourg-en-Cotentin       77808
43                     Antibes       75130
47221                  Ajaccio       73822
3404                    Cannes       73255
49941            Saint-Nazaire       72057
93951                   Calais       67380
113291                  Fréjus       57082
19712                 Narbonne       56395
114963                  Hyères       55103
2343            Cagnes-sur-Mer       52580
69195               Saint-Malo       47323
66555                     Sète       44712
119476    Six-Fours-les-Plages       36203
38678            Saint-Raphaël       35950
16448     Saint-Laurent-du-Var       30941
60375                     Agde       29103
105710                  Dieppe       28358
17145      

In [51]:
# Filtrer 'base' pour ne conserver que les communes présentes dans 'top_40_communes'
top_40_communes_names = top_40_communes['nom_commune'].unique()  # Liste des noms de communes les plus peuplées
filtered_base = base[base['nom_commune'].isin(top_40_communes_names)]

# Grouper par 'nom_commune' et 'identifiant_tri', puis compter le nombre d'occurrences
zone_counts = filtered_base.groupby(['nom_commune', 'identifiant_tri']).size().reset_index(name='nombre_occurences')

# Ajouter la population des communes à 'zone_counts' en fusionnant avec 'top_40_communes'
zone_counts = pd.merge(zone_counts, top_40_communes[['nom_commune', 'Population']], on='nom_commune', how='left')

# Trier par population des communes (en ordre décroissant)
tri_by_commune = zone_counts.sort_values(by='Population', ascending=False) \
    .drop(columns=['Population']) \
    .reset_index(drop=True)

# Afficher le résultat trié
display(tri_by_commune)


,nom_commune,identifiant_tri,nombre_occurences
0,Nice,FRD_TRI_NICE,985
1,Bordeaux,FRF_TRI_BORD,330
2,Toulon,FRD_TRI_TOULON,633
3,Caen,FRH_TRI_CAEN,39
4,Dunkerque,FRA_TRI_DUNKERQUE,129
5,Cherbourg-en-Cotentin,FRH_TRI_CHERBOURG_OCTEVILLE,132
6,Antibes,FRD_TRI_NICE,85
7,Ajaccio,FRE_TRI_AJACCIO,2
8,Cannes,FRD_TRI_NICE,131
9,Saint-Nazaire,FRD_TRI_PERPIGNAN,1


In [52]:
tri = tri_by_commune.loc[tri_by_commune['nom_commune'] == 'Nice']

In [53]:
tri = tri_by_commune.loc[tri_by_commune['nom_commune'] == 'Nice', 'identifiant_tri'].iloc[0]

In [54]:
gdf = gdf.drop(columns=['id','dept'])

In [55]:
show(gdf)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
C:\Users\flori\AppData\Local\Temp\ipykernel_15692\209050519.py:1: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\AppData\Local\Temp\ipykernel_15692\209050519.py:1: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\AppData\Local\Temp\ipykernel_15692\209050519.py:1: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\AppData\Local\Temp\ipykernel_15692

In [56]:
identifiant_tri = tri_by_commune.loc[tri_by_commune['nom_commune'] == 'Nice', 'identifiant_tri'].iloc[0]


In [57]:
print(base['beach_coordinates'].dtype)
isinstance(base['beach_coordinates'], list)

object


False

In [58]:
# Fonction de conversion forcée pour transformer les coordonnées sous forme de liste de tuples (latitude, longitude)
def force_convert_to_tuple_list(beach_coordinates):
    try:
        # Si c'est déjà une liste, la convertir en liste de tuples
        if isinstance(beach_coordinates, list):
            # Convertir chaque élément de la liste en tuple (latitude, longitude)
            return [(float(coord[0]), float(coord[1])) for coord in beach_coordinates]

        # Si c'est une chaîne représentant une liste, on évalue la chaîne en une liste de tuples
        elif isinstance(beach_coordinates, str):
            coords = eval(beach_coordinates)  # Utilisation de eval avec prudence
            return [(float(coord[0]), float(coord[1])) for coord in coords]
        
        # Si c'est un tuple contenant deux éléments (latitude, longitude), on le convertit en liste
        elif isinstance(beach_coordinates, tuple) and len(beach_coordinates) == 2:
            return [(float(beach_coordinates[0]), float(beach_coordinates[1]))]
        
        # Si la donnée n'est pas dans un format attendu, retourner une liste vide
        else:
            return []

    except Exception as e:
        print(f"Erreur lors de la conversion des coordonnées : {e}")
        return []

# Forcer la conversion de la colonne 'beach_coordinates' en liste de tuples (latitude, longitude)
base['beach_coordinates'] = base['beach_coordinates'].apply(force_convert_to_tuple_list)

# Vérification de la conversion
print(base['beach_coordinates'].head())  # Afficher les premières lignes pour vérifier


15    [(43.197136, 3.214825), (43.1914356, 3.2078971)]
43                             [(43.553796, 7.134512)]
44                             [(43.553796, 7.134512)]
45                             [(43.553796, 7.134512)]
46                             [(43.553796, 7.134512)]
Name: beach_coordinates, dtype: object


In [59]:
# Créer un DataFrame avec les occurrences uniques de 'nom_commune' et 'beach_coordinates'
unique_beach_coordinates_df = base[['nom_commune', 'beach_coordinates']].drop_duplicates(subset=['nom_commune'])

# Afficher le DataFrame pour vérifier le résultat
print(unique_beach_coordinates_df)


                   nom_commune  \
15                      Fleury   
43                     Antibes   
2343            Cagnes-sur-Mer   
3404                    Cannes   
6408      Mandelieu-la-Napoule   
...                        ...   
124168     Notre-Dame-de-Monts   
124514   Saint-Hilaire-de-Riez   
125025     Saint-Jean-de-Monts   
125443  Saint-Michel-en-l'Herm   
125776                 Triaize   

                                        beach_coordinates  
15       [(43.197136, 3.214825), (43.1914356, 3.2078971)]  
43                                [(43.553796, 7.134512)]  
2343    [(43.6510615, 7.1533523), (43.6518628, 7.15450...  
3404     [(43.5357063, 7.0365724), (43.550035, 6.995074)]  
6408      [(43.502951, 6.9545446), (43.522677, 6.941767)]  
...                                                   ...  
124168                                                 []  
124514                           [(46.722042, -1.980331)]  
125025  [(47.0010281, -2.2981658), (46.8131958, -2.

In [60]:
print(len(base['beach_coordinates'].iloc[1]))
beach_coordinates = unique_beach_coordinates_df[unique_beach_coordinates_df['nom_commune'] == commune_name]['beach_coordinates'].values
print(beach_coordinates.dtype)


1
object


In [68]:
beach_coordinates = unique_beach_coordinates_df[unique_beach_coordinates_df['nom_commune'] == 'Nice']['beach_coordinates']
print(beach_coordinates.size) 
# print(len(base['beach_coordinates'].iloc[1]))
# u=beach_coordinates.iloc[0]
# print(type(u))
# print(type(u[3]))


1


In [ ]:
import folium
from folium.plugins import HeatMap
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
from pyproj import Transformer
from IPython.display import IFrame
import os

# Créer un transformer pour convertir les coordonnées (Lambert-93 -> Latitude/Longitude)
transformer = Transformer.from_crs("EPSG:2154", "EPSG:4326", always_xy=True)

# Fonction de transformation des coordonnées
def transform_coordinates(x, y):
    lon, lat = transformer.transform(x, y)  # Lambert-93 vers WGS84
    return lat, lon  # Retourne sous forme (latitude, longitude)

# Fonction pour créer et sauvegarder une carte
def create_map(gdf, commune_name, price_data, identifiant_tri, beach_coordinates):
    # Filtrer les données de `gdf` pour la commune via `identifiant_tri`
    commune_data = gdf[gdf['id_tri'] == identifiant_tri]
    
    # Vérifier si les données sont présentes pour l'identifiant
    if commune_data.empty:
        print(f"Pas de données pour l'identifiant_tri : {identifiant_tri}")
        return None

    # Extraire les coordonnées de la mairie via `price_data`
    price_data_commune = price_data[price_data['nom_commune'] == commune_name]
    if price_data_commune.empty:
        print(f"Pas de données de prix pour la commune : {commune_name}")
        return None
    
    latitude_mairie = price_data_commune['latitude_mairie'].iloc[0]
    longitude_mairie = price_data_commune['longitude_mairie'].iloc[0]

    # Vérifier si les coordonnées de la mairie ne sont pas NaN
    if pd.isna(latitude_mairie) or pd.isna(longitude_mairie):
        print(f"Coordonnées de la mairie manquantes pour : {commune_name}")
        return None

    # Créer une carte centrée sur la mairie
    m = folium.Map(location=[latitude_mairie, longitude_mairie], zoom_start=12)
    
    # Ajouter les polygones inondables sans remplissage de couleur
    for _, row in commune_data.iterrows():
        if isinstance(row['geometry'], Polygon):
            coordinates = [(x, y) for x, y in row['geometry'].exterior.coords]
            transformed_coords = [transform_coordinates(x, y) for x, y in coordinates]
            folium.Polygon(
                locations=transformed_coords,
                color='blue',  # Garder le contour bleu
                weight=2,
                fill=False,  # Pas de remplissage
            ).add_to(m)
    
    # Ajouter une heatmap des prix
    price_data_commune = price_data_commune.dropna(subset=['latitude', 'longitude', 'prix_m2'])
    heat_data = price_data_commune[['latitude', 'longitude', 'prix_m2']].values.tolist()
    
    if heat_data:
        HeatMap(heat_data, radius=10).add_to(m)

    # Ajouter les points de plage avec des carrés rouges à partir de `beach_coordinates` dans `base`
    for coord in beach_coordinates:
        # Vérifier si l'objet coord contient des attributs lat, lon
        if isinstance(coord, tuple) and len(coord) == 2:
            lat, lon = coord  # extraire latitude et longitude
            print([lat,lon])
            folium.Marker(
                location=[lat, lon],
                icon=folium.Icon(color='red', icon='square', icon_size=(50,50))  # Carré rouge
            ).add_to(m)

    return m

# Sauvegarder les cartes dans "maps/"
maps = {}
for _, row in tri_by_commune.iterrows():
    commune_name = row['nom_commune']
    identifiant_tri = row['identifiant_tri']
    
    # Extraire les coordonnées des plages pour la commune depuis 'base'
    beach_coordinates = unique_beach_coordinates_df[unique_beach_coordinates_df['nom_commune'] == commune_name]['beach_coordinates']

    if beach_coordinates.size > 0:
        beach_coordinates = beach_coordinates.iloc[0]  # Liste de tuples (latitude, longitude)
    
    # Créer une carte pour chaque commune
    m = create_map(gdf, commune_name, base, identifiant_tri, beach_coordinates)
    if m:
        # Créer le répertoire "maps" s'il n'existe pas
        os.makedirs('maps', exist_ok=True)
        # Sauvegarder la carte dans un fichier HTML dans le répertoire "maps"
        file_name = f'maps/{commune_name}_map.html'
        m.save(file_name)
        maps[commune_name] = file_name
        print(f"Carte pour {commune_name} enregistrée sous : {file_name}")

# Afficher les cartes dans des iframes pour visualisation dans le notebook
for commune, file_name in maps.items():
    display(IFrame(file_name, width=600, height=400))


[43.7812092, 7.5102456]
[43.9434372, 7.1947915]
[43.687128, 7.3226097]
[43.9553277, 6.8598036]
Carte pour Nice enregistrée sous : maps/Nice_map.html
[44.7851745, -0.7054801]
[44.8029523, -0.5361902]
[44.828132, -0.586971]
Carte pour Bordeaux enregistrée sous : maps/Bordeaux_map.html
[43.0741109, 5.8838953]
[43.0675548, 5.8068092]
[43.05814, 5.863129]
[43.0610204, 5.8661124]
[43.115101, 6.220726]
[43.1326192, 5.7802263]
[43.0019142, 6.3730119]
[42.9923375, 6.2194056]
Carte pour Toulon enregistrée sous : maps/Toulon_map.html
Carte pour Caen enregistrée sous : maps/Caen_map.html
[51.048457, 2.383567]
Carte pour Dunkerque enregistrée sous : maps/Dunkerque_map.html
[49.666568, -1.69688]
Carte pour Cherbourg-en-Cotentin enregistrée sous : maps/Cherbourg-en-Cotentin_map.html
[43.553796, 7.134512]
Carte pour Antibes enregistrée sous : maps/Antibes_map.html
Pas de données pour l'identifiant_tri : FRE_TRI_AJACCIO
[43.5357063, 7.0365724]
[43.550035, 6.995074]
Carte pour Cannes enregistrée sous : 